In [7]:

import sqlite3


conn = sqlite3.connect('inventory.db')
conn.row_factory = sqlite3.Row
cursor = conn.cursor()


# Ensure that the inventory table exists
cursor.executescript('''
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY autoincrement,
    barcode TEXT,
    location TEXT,
    name TEXT NOT NULL,
    UNIQUE (id, name)
);

CREATE TABLE if not exists checkout_log (
    id INTEGER PRIMARY KEY autoincrement, 
    checkoutTime DATETIME DEFAULT CURRENT_TIMESTAMP,
    person text NON NULL,
    itemID INTEGER NON NULL, 
    returned BOOL DEFAULT FALSE
);

CREATE TABLE if not exists users (
    userId INTEGER primary key autoincrement,
    actualName text unique,
    password text unique 
);

CREATE TABLE if not exists user_log (
    id INTEGER primary key autoincrement,
    theTime DATETIME DEFAULT CURRENT_TIMESTAMP,
    userId integer,
    action text,
    authenticated BOOL
);
''')
cursor.connection.commit()


In [46]:
def add_user(name, password):
    cursor.execute("INSERT INTO users (actualName, password) VALUES (?, ?)", (name, password))
    conn.commit()

def check_login(password):
    row = cursor.execute("SELECT userId from users where password = ? LIMIT 1;", [password]).fetchone()
    if row is None:
        return None
    
    return row["userId"]
    
    
def add_item(name, location=None, barcode=None):
    cursor.execute("INSERT into inventory (location, name, barcode) VALUES (?, ?, ?)", (location, name, barcode))
    conn.commit()


def set_location(id, location):
    cursor.execute("UPDATE inventory SET location = ? WHERE id = ?", (location, id))
    conn.commit()

def use_item(id, person):
    cursor.execute("INSERT into checkout_log (itemID, person, returned) VALUES (?, ?, false)", (id, person))
    conn.commit()

def return_item(id, person):
    cursor.execute("INSERT into checkout_log (itemID, person, returned) VALUES (?, ?, true)", (id, person))
    conn.commit()
    
    
def set_location_barcode(barcode, location):
    cursor.execute("UPDATE inventory SET location = ? WHERE barcode = ?", (location, barcode))
    conn.commit()

def use_item_barcode(barcode, person):
    cursor.execute("INSERT into checkout_log (itemID, person, returned) VALUES ((SELECT inventory.id from inventory where barcode = ?), ?, false)", (barcode, person))
    conn.commit()

def return_item_barcode(barcode, person):
    cursor.execute("INSERT into checkout_log (itemID, person, returned) VALUES ((SELECT inventory.id from inventory where barcode = ?), ?, false)", (barcode, person))
    conn.commit()
    
def set_barcode(itemId, barcode):
    cursor.execute("UPDATE inventory set barcode = ? where id = ?", (barcode, itemId))
    conn.commit()
    
def get_all_items():
    rows = cursor.execute("with itemsReturned as (\n    select max(id) as max_id, itemID, not(returned) as in_use\n    from checkout_log\n    group by itemID)\n\n\nselect inventory.id, barcode, location, name, IFNULL(in_use, FALSE) as in_use\nfrom inventory left outer join itemsReturned on inventory.id = itemID;").fetchall()
    return [dict(i) for i in rows]

In [12]:
add_item("world")
use_item(1, "Jesse")
return_item(1, "Jesse")

In [33]:
get_all_items()

# conn.close()

[{'id': 1, 'barcode': None, 'location': None, 'name': 'world', 'in_use': 0}]

In [34]:
use_item(4, "Jesse")


In [35]:
add_item("hello")


In [36]:
return_item(4, "Fooey")

In [48]:
check_login("hello")

1

In [38]:
add_user("jesse", "hello")

IntegrityError: UNIQUE constraint failed: users.actualName

In [ ]:


conn.close()